# J. Paul Getty Museum Collection Download
The museum is sharing an art collection dataset with some CC0 license, what we can use to train our CC0 ResNet backbone for other projects!
Their docs page: https://data.getty.edu/museum/collection/docs/ contains documentation how to download the images from their API.
We can use the image as long as it has a valid '"id": "https://creativecommons.org/publicdomain/zero/1.0/"' field in its data, what we will filter on. The dataset may contain up to 80000 images, and we will need a lot of that to train a ResNet backbone that would have transferable knowledge.

### Constants and imports

In [ ]:
import requests
import os
import random
import json
from tqdm.notebook import tqdm
import pandas as pd
from atomicwrites import atomic_write

CMA_GITHUB_DATA_CSV = 'https://github.com/ClevelandMuseumArt/openaccess/raw/master/data.csv'
METADATA_DIRECTORY = 'metadata'
DATASET_NAME = 'get'
RAW_METADATA_DIRECTORY = os.path.join(METADATA_DIRECTORY, 'raw')
RAW_METADATA_DIRECTORY_FILENAME = os.path.join(RAW_METADATA_DIRECTORY, f'{DATASET_NAME}.csv')
RAW_IMAGE_DIRECTORY = os.path.join('dataset', 'raw')
DATASET_IMAGES_LIMIT = 70000
DOWNLOADING_SAVE_PERIOD = 5

In [ ]:
PREFIX http: <http://www.w3.org/2011/http#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT DISTINCT ?type WHERE {
  ?sub rdf:type ?type .
} LIMIT 30